
Autograd: automatic differentiation
===================================

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our
first neural network.


The ``autograd`` package provides automatic differentiation for all operations
on Tensors. It is a define-by-run framework, which means that your backprop is
defined by how your code is run, and that every single iteration can be
different.

Let us see this in more simple terms with some examples.

Tensor
--------

``torch.Tensor`` is the central class of the package. If you set its attribute
``.requires_grad`` as ``True``, it starts to track all operations on it. When
you finish your computation you can call ``.backward()`` and have all the
gradients computed automatically. The gradient for this tensor will be
accumulated into ``.grad`` attribute.

To stop a tensor from tracking history, you can call ``.detach()`` to detach
it from the computation history, and to prevent future computation from being
tracked.

To prevent tracking history (and using memory), you can also wrap the code block
in ``with torch.no_grad():``. This can be particularly helpful when evaluating a
model because the model may have trainable parameters with `requires_grad=True`,
but we don't need the gradients.

There’s one more class which is very important for autograd
implementation - a ``Function``.

``Tensor`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each variable has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Tensor`` (except for Tensors created by the user - their
``grad_fn is None``).

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Tensor``. If ``Tensor`` is a scalar (i.e. it holds a one element
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``gradient``
argument that is a tensor of matching shape.



In [24]:
import torch

Create a tensor and set requires_grad=True to track computation with it



In [25]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[ 1.,  1.],
        [ 1.,  1.]])


Do an operation of tensor:



In [26]:
y = x + 2
print(y)

tensor([[ 3.,  3.],
        [ 3.,  3.]])


``y`` was created as a result of an operation, so it has a ``grad_fn``.



In [27]:
print(y.grad_fn)

Do more operations on y



In [32]:
z = y * y * 3
out = z.mean()

print(z, out)
print(out.grad_fn)

tensor(1.00000e+06 *
       [ 4.3318,  4.0754,  0.3366]) tensor(1.00000e+06 *
       2.9146)


Gradients
---------
Let's backprop now.


In [29]:
out.backward()

print gradients d(out)/dx




In [30]:
print(x.grad)

tensor([[ 4.5000,  4.5000],
        [ 4.5000,  4.5000]])


You should have got a matrix of ``4.5``. Let’s call the ``out``
*Tensor* “$o$”.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



You can do many crazy things with autograd (try do the following with Tensorflow)!



In [31]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 1201.6389, -1165.5265,   334.9677])


In [33]:
gradients = torch.tensor([1.0, 0.1, 0.001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([ 512.0000,   51.2000,    0.5120])


Here `gradients` come from higher layers in your neural network i.e. if $l$ is your loss function then `gradients` is given by $$\left[\frac{\partial l}{\partial y_1}, \frac{\partial l}{\partial y_2}, \frac{\partial l}{\partial y_3}\right]$$ You can also stop autograd from tracking history on Tensors
with requires_grad=True by wrapping the code block in
``with torch.no_grad():``



In [34]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False


**Read Later:**

Documentation of ``autograd`` and ``Function`` is at
http://pytorch.org/docs/autograd



Exercise 1
------------

1. Use `torch.randn` to create a tensor `x` of size (10, 5).
2. Use `torch.randn` with `requires_grad=True` to create tensors `W` and `b` of sizes (5, 2) and (2,) respectively.
3. Using `W` and `b` compute `z` the output of a linear layer with weights `W` and biases `b`.
4. Create a tensor `dloss` of size (10, 2) with a value of 1/10 in each component.
5. Use `z.backward(dloss)` to find the gradients with respect to `W` and `b`.
6. Varify analytically that these are the the correct gradients.

In [1]:
# Solution
# x = torch.randn(10, 5)
# W = torch.randn(5, 2, requires_grad=True)
# b = torch.randn(2, requires_grad=True)
# z = torch.matmul(x, W) + b
# dloss = torch.ones_like(z) / 10
# z.backward(dloss)
# print(W.grad)
# print(b.grad)

$$
\frac{\partial l}{\partial W_{n,m}} = \sum_{i, j} \frac{\partial l}{\partial z_{i,j}}\frac{\partial z_{i, j}}{w_{n,m}} = \sum_{i} \frac{1}{10} x_{i, m}
$$
$$
\frac{dl}{dW_{:, m}} = x.mean(dim=0)
$$
$$
\frac{\partial l}{\partial b_n} = \sum_{i, j} \frac{\partial l}{\partial z_{i, j}} \frac{\partial z_{i, j}}{\partial b_n} = \sum_i \frac{1}{10} = 1
$$

In [2]:
# print(x.mean(dim=0))

Exercise 2
------------

1. Use `torch.randn` to create a tensor `x` of size (5,).
2. Use `torch.randn` with `requires_grad=True` to create a tensor `W` size (5,).
3. Using `torch.sin` compute the sin function of the element-wise product of `x` and `W`.
4. Find the sum `z` of the tensor in 3.
5. Use `z.backward()` to find the gradients with respect to `W`.
6. Varify analytically that these are the the correct gradients.

In [21]:
# Solution
# import torch
# x = torch.randn(5)
# W = torch.randn(5, requires_grad=True)
# z = torch.sin(W * x).sum()
# z.backward()
# print(W.grad)

$$
\frac{\partial{z}}{\partial W_i} = \frac{\partial \sin(W_i x_i)}{\partial W_i} = x_i \cos(W_i x_i)
$$
$$
\frac{\partial{z}}{\partial W} = x \cos(W x)
$$

In [22]:
# print(x * torch.cos(W * x))